# Part 1

In [1]:
filename = "inputs/12-22.txt"
with open(filename, "r") as f:
    data = f.read()

In [2]:
from collections import deque
import math
import re

In [3]:
map_, path = data.split("\n\n")
path = [(int(l), -90 if r == "R" else 90 if r == "L" else 0) for l, r in re.findall("([0-9]+)([RL]?)", path)]
map_ = map_.splitlines()
my = len(map_)
mx = max([len(l) for l in map_])
m = {}
for j, l in enumerate(map_):
    for i, c in enumerate(l):
        if c != " ":
            m[i, j] = c

In [4]:
def apply_isometry(p, rotation_angle=0, rotation_center=(0, 0), translation=(0, 0)):
    """Apply a translation followed by a rotation."""
    x, y = p
    dx, dy = translation
    x += dx
    y += dy
    cx, cy = rotation_center
    x, y = x - cx, y - cy
    if rotation_angle == -90:  # R
        x, y = -y, x
    if rotation_angle == 90:  # L
        x, y = y, -x
    if rotation_angle == 180:
        x, y = -x, -y
    return round(cx + x), round(cy + y)

def move(x, y, dx, dy, m=m, mx=mx, my=my):
    tx = (x + dx) % mx
    ty = (y + dy) % my
    while not (tx, ty) in m:
        tx = (tx + dx) % mx
        ty = (ty + dy) % my
    return tx, ty, dx, dy

In [5]:
def get_password(m, move_func):
    orientations = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    for p in sorted(m.keys(), key=lambda t: (t[1], t[0])):
        if m[p] == ".":
            start = p
            break
    x, y = start
    dx, dy = (1, 0)
    for length, angle in path:
        for _ in range(length):
            tx, ty, dtx, dty = move_func(x, y, dx, dy)
            if m[tx, ty] == "#":
                break
            else:
                x, y, dx, dy = tx, ty, dtx, dty
        dx, dy = apply_isometry((dx, dy), angle)

    return 1000 * (y + 1) + 4 * (x + 1) + orientations.index((dx, dy))

print(get_password(m, move))

196134


# Part 2

In [6]:
if mx <= 16:
    cx, cy = (4, 4)
else:
    cx, cy = (50, 50)
sides = deque()
for i in range(mx//cx):
    for j in range(my//cy):
        if (i * cx, j * cy) in m:
            sides.appendleft((i, j))

mi = max([i for i, _ in sides]) + 1
mj = max([j for _, j in sides]) + 1

In [7]:
# dict of top-left, top-right, botom-right, bottom-left corners 3D coordinates
coordinates = {sides.pop(): ((0, 0, 0), (1, 0, 0), (1, 1, 0), (0, 1, 0))}

while sides:
    (i, j) = sides.pop()
    adjacent = False
    for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
        if (i + di, j + dj) in coordinates:
            # faces share an edge on the map
            adjacent = True
            tl, tr, br, bl = coordinates[i + di, j + dj]
            x, y, z = zip(tl, tr, br, bl)
            dx = 0 if min(x) != max(x) else 1 - 2 * min(x)
            dy = 0 if min(y) != max(y) else 1 - 2 * min(y)
            dz = 0 if min(z) != max(z) else 1 - 2 * min(z)
            if di == -1:
                ftl = tr
                fbl = br
                x, y, z = ftl
                ftr = x + dx, y + dy, z + dz 
                x, y, z = fbl
                fbr = x + dx, y + dy, z + dz
            if di == 1:
                ftr = tl
                fbr = bl
                x, y, z = ftr
                ftl = x + dx, y + dy, z + dz
                x, y, z = fbr
                fbl = x + dx, y + dy, z + dz
            if dj == -1:
                ftl = bl
                ftr = br
                x, y, z = ftr
                fbr = x + dx, y + dy, z + dz
                x, y, z = ftl
                fbl = x + dx, y + dy, z + dz
            if dj == 1:
                fbr = tr
                fbl = tl
                x, y, z = fbl
                ftl = x + dx, y + dy, z + dz
                x, y, z = fbr
                ftr = x + dx, y + dy, z + dz
            coordinates[i, j] = (ftl, ftr, fbr, fbl)
    if not adjacent:
        sides.appendleft((i, j))

In [8]:
def compute_isometry(a, b, ta, tb):
    ax, ay = a
    bx, by = b
    tax, tay = ta
    tbx, tby = tb
    angle = math.atan2((tay - tby) * (bx - ax) - (tbx - tax) * (ay - by), (tbx - tax) * (bx - ax) + (tay - tby) * (ay - by))
    deg_angle = round(angle * 180 / math.pi)
    if deg_angle == -180:
        deg_angle = 180
    if deg_angle != 0:
        cx = round((-math.cos(angle) * (ax + tax) + math.sin(angle) * (-ay + tay) + ax + tax)/(2 - 2 * math.cos(angle)), 1)
        cy = -round((math.sin(angle) * (tax - ax) - math.cos(angle) * (-ay - tay) - ay - tay)/(2 - 2 * math.cos(angle)), 1)
        return (cx, cy), deg_angle, (0, 0)
    else:
        return (0, 0), 0, (tbx - bx, tby - by)

def compute_isometries(sides, cx=cx, cy=cy):
    isometries = {}
    for i, j in sides:
        isometries[i, j] = {}
        tl, tr, br, bl = sides[i, j]
        top = j * cy
        bottom = (j + 1) * cy - 1
        left = i * cx
        right = (i + 1) * cx - 1
        for di, dj, a, b, oa, ob in (
            (-1, 0, tl, bl, (left - 1, top), (left - 1, bottom)),
            (1, 0, tr, br, (right + 1, top), (right + 1, bottom)),
            (0, -1, tl, tr, (left, top - 1), (right, top - 1)),
            (0, 1, bl, br, (left, bottom + 1), (right, bottom + 1))
        ):
            if (i + di, j + dj) not in sides:
                # if the edge is a cut on the map
                # find the face that shares edge [a, b]
                # and the transformation to apply in the original map coordinate system
                for f in sides:
                    if f == (i, j):
                        continue
                    pts = sides[f]
                    if len(set(pts).intersection((a, b))) == 2:
                        ti, tj = f
                        top = tj * cy
                        bottom = (tj + 1) * cy - 1
                        left = ti * cx
                        right = (ti + 1) * cx - 1
                        ot = [(left, top), (right, top), (right, bottom), (left, bottom)]
                        isometries[i, j][di, dj] = compute_isometry(oa, ob, ot[pts.index(a)], ot[pts.index(b)])
    return isometries

isometries = compute_isometries(coordinates)

In [9]:
def cube_move(x, y, dx, dy, m=m, mi=mi, mj=mj):
    tx = x + dx
    ty = y + dy
    if (tx, ty) not in m:
        # warp
        i = (x // cx) % mi
        j = (y // cy) % mj
        c, rot, tr = isometries[i, j][dx, dy]
        # compute new coordinates
        tx, ty = apply_isometry((tx, ty), rot, c, tr)
        dx, dy = apply_isometry((dx, dy), rot)
    return tx, ty, dx, dy

In [10]:
print(get_password(m, cube_move))

146011
